# Синтаксический анализ

Парсинг (синтаксический анализ) представляет собой процесс сопоставления последовательности слов или символов — так называемой формальной грамматике.

Посмотрим, как в [формате БНФ](https://en.wikipedia.org/wiki/Backus%E2%80%93Naur_form) описывается в python оператор ветвления. https://docs.python.org/3/reference/compound_stmts.html#if 
```
if_stmt ::=  "if" assignment_expression ":" suite
             ("elif" assignment_expression ":" suite)*
             ["else" ":" suite
```
Формальная грамматика состоит из алфавита (набор символов) и правил порождения. То есть определение, "новое понятие - это вот такая композиция из уже введенных понятий".

При разборе текст разбивается на единицы (токены). У каждого токена есть определение что это такое. Терминальный токен - он состоит из символов алфавита.

Для синтаксического разбора можно использовать пакет [pyparsing](https://pythonhosted.org/pyparsing/pyparsing-module.html). Покажем как им пользоваться на примерах.

Документация: https://pyparsing-docs.readthedocs.io/en/latest/]


In [1]:
# считаем, что этот импорт уже выполнен во всех примерах
import pyparsing as pp

## Пример 1. Разбор строки import

Дан текст вида
```python
import matplotlib.pyplot as plt
```
Хотим привести его в вид
```python
{ 'import': [ 'matplotlib', 'pyplot' ], 'as': 'plt' }
```

In [57]:
module_name = pp.Word(pp.alphas + '_')                              # имя модуля - это латинские буквы и _
full_module_name = module_name + pp.ZeroOrMore('.' + module_name)   # модуль и может быть через . еще имена модулей
import_as = pp.Optional('as' + module_name)                         # может быть, а может и не быть часть as алиас
parse_module = 'import' + full_module_name + import_as              # соберем все в единую строку, начинающуюся с import

In [64]:
# разберем с помощью этих правил строку
text = 'import        matplotlib.pyplot     as      plt'
parse_module.parseString(text)

ParseResults(['import', 'matplotlib', '.', 'pyplot', 'as', 'plt'], {})

In [59]:
# или в виде списка
parse_module.parseString(text).asList()

['import', 'matplotlib', '.', 'pyplot', 'as', 'plt']

## Suppress - уберем найденные токены из результирующего списка

То есть эти токены обязаны быть, но в результат они не войдут. Варианты использования (пример для разделяющей запятой)
```python
comma = Suppress( Literal(",") ) 
comma = Literal(",").suppress() 
comma = Suppress(",")
```

In [65]:
module_name = pp.Word(pp.alphas + '_')                              # имя модуля - это латинские буквы и _
full_module_name = module_name + pp.ZeroOrMore(pp.Suppress('.') + module_name)   # модуль и может быть через . еще имена модулей
import_as = pp.Optional(pp.Suppress('as') + module_name)                         # может быть, а может и не быть часть as алиас
parse_module = pp.Suppress('import') + full_module_name + import_as              # соберем все в единую строку, начинающуюся с import

text = 'import matplotlib.pyplot as plt'
parse_module.parseString(text).asList()

['matplotlib', 'pyplot', 'plt']

### Дадим имя правилу и будем обращаться по имени

* Все выражение справа заключим в `( )`
* После него поставим `('имя')`
* Можно обращаться по имени `.имя` к части разобранного выражения

In [41]:
module_name = pp.Word(pp.alphas + '_')
full_module_name = (module_name + pp.ZeroOrMore(pp.Suppress('.') + module_name)) ('modules')  # дали имя modules
import_as = (pp.Optional(pp.Suppress('as') + module_name))('import_as')                       # дали имя import_as
parse_module = pp.Suppress('import') + full_module_name + import_as

text = 'import matplotlib.pyplot as plt'
res = parse_module.parseString(text)
print(f'{res=}')
print(res.modules.asList())
print(res.import_as.asList())

res=ParseResults(['matplotlib', 'pyplot', 'plt'], {'modules': ['matplotlib', 'pyplot'], 'import_as': ['plt']})
['matplotlib', 'pyplot']
['plt']


## setParseAction - преобразуем в нужный формат через lambda

In [43]:
parse_module = (pp.Suppress('import') + full_module_name).setParseAction(lambda t: {'import': t.modules.asList(), 'as': t.import_as.asList()[0]})

In [49]:
module_name = pp.Word(pp.alphas + '_')
full_module_name = (module_name + pp.ZeroOrMore(pp.Suppress('.') + module_name)) ('modules')  # дали имя modules
import_as = (pp.Optional(pp.Suppress('as') + module_name))('import_as')                       # дали имя import_as
parse_module = (pp.Suppress('import') + full_module_name + import_as).setParseAction(lambda t: {'import': t.modules.asList(), 'as': t.import_as.asList()[0]})
parse_module.parseString(text)

ParseResults([{'import': ['matplotlib', 'pyplot'], 'as': 'plt'}], {})

In [50]:
parse_module.parseString(text).asList()

[{'import': ['matplotlib', 'pyplot'], 'as': 'plt'}]

In [51]:
parse_module.parseString(text).asList()[0]

{'import': ['matplotlib', 'pyplot'], 'as': 'plt'}

### Количество символов

Для указания точного количества символов используют аргументы [Word](https://pythonhosted.org/pyparsing/pyparsing.Word-class.html)

`__init__(self, initChars, bodyChars=None, min=1, max=0, exact=0, asKeyword=False, excludeChars=None)`       ''')

In [66]:
# US postal code can be a 5-digit zip, plus optional 4-digit qualifier
zip = pp.Combine(pp.Word(pp.nums, exact=5) + pp.Optional('-' + pp.Word(pp.nums, exact=4)))
zip.runTests('''
   # traditional ZIP code
   12345
   
   # ZIP+4 form
   12101-0001
   
   # invalid ZIP
   98765-
   ''')



# traditional ZIP code
12345
['12345']

# ZIP+4 form
12101-0001
['12101-0001']

# invalid ZIP
98765-
98765-
     ^
ParseException: Expected end of text, found '-'  (at char 5), (line:1, col:6)
FAIL: Expected end of text, found '-'  (at char 5), (line:1, col:6)


(False,
 [('12345', ParseResults(['12345'], {})),
  ('12101-0001', ParseResults(['12101-0001'], {})),
  ('98765-', Expected end of text, found '-'  (at char 5), (line:1, col:6))])

## Пробельные символы, начало и конец

По умолчанию все пробельные символы, начало и конец строки игнорируются. Чтобы их подключить, указывайте явно

* [leaveWhitespace()](https://pythonhosted.org/pyparsing/pyparsing.Forward-class.html#leaveWhitespace) - Disables the skipping of whitespace before matching the characters in the ParserElement's defined pattern. This is normally only used internally by the pyparsing module, but may be needed in some whitespace-sensitive grammars.
* [White()](https://pythonhosted.org/pyparsing/pyparsing.White-class.html) - пробельный символ (пробел, \n, \t, \r)
* AtLineStart - точно в начале строки
* AtLineEnd - точно в конце строки
* LineStart - 
Matches if current position is at the beginning of a line within the parse strin
*  	LineE - d
Matches if current position is at the end of a line within the parse str
* 
 	StringS - art
Matches if current position is at the beginning of the parse s
* ng
 	Str - ngEnd
Matches if current position is at the end of the parse
* ring
 	W - rdStart
Matches if the current position is at the beginning of a Word, and is not preceded by any character in a given set of wordChars (default=prin* bles). -  	WordEnd
Matches if the current position is at the end of a Word, and is not followed by any character in a given set of wordChars (default=printables).

In [55]:
(pp.LineStart() + pp.Word(pp.alphas)).parseString("ABC")    # passes
(pp.LineStart() + pp.Word(pp.alphas)).parseString("  ABC")  # passes
pp.AtLineStart(pp.Word(pp.alphas)).parseString("ABC")     # passes
pp.AtLineStart(pp.Word(pp.alphas)).parseString("  ABC")     # fails

ParseException: Expected W:(A-Za-z), found ' '  (at char 0), (line:1, col:1)

## restOfLine = rest of line

## Unicode символы

Специальные наборы алфавитов. Но! Нам это вообще не нужно.


In [56]:
# функция для нормальной печати юникодных символов при разборе
def bprint(obj):
    print(obj.__repr__().decode('string_escape'))


# Задача

Разобрать h2 заголовок с разметкой типов заголовка, default = TEXT, добавить SKIP как возможность двух типов.

## Документация

* [Индекс](https://pythonhosted.org/pyparsing/identifier-index.html) фукнций и классов
* http://xgu.ru/wiki/pyparsing - маленькая страница, но на русском
* http://netsago.org/ru/docs/1/8/ - на русском, разбор химических формул (возможно, перевод)
* http://s.arboreus.com/2009/07/easy-parsing-in-python.html - на русском, пример входных данных х и у из таблицы, Group
* https://habr.com/ru/articles/239081/ - статья на Хабре - разбор примера на import
* https://habr.com/ru/articles/241670/ - продолжение статьи на Хабре, примеры с размерностями русскими и степенями.

## Вопросы для самопроверки

1. Чем отличается Optional от ZeroOrMore?
2. Какие названия методов являются устаревшими? Optional, optional, |
3. Чем отличается Group от Combine?